In [ ]:
# default_exp label

# Display

> A collection of functions to do label-based quantification

In [ ]:
#hide
from nbdev.showdoc import *

## Tag Search

The tag search is implemented based on the compare_frags from the search. 
As the number of reporter channels is known, we


Additional resources: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_IsobaricAnalyzer.html

https://assets.thermofisher.com/TFS-Assets/CMD/Reference-Materials/PP-TMT-Multiplexed-Protein-Quantification-HUPO2015-EN.pdf


https://assets.thermofisher.com/TFS-Assets/CMD/Reference-Materials/PP-TMT-Multiplexed-Protein-Quantification-HUPO2015-EN.pdf

In [ ]:
#export
from numba import njit
from alphapept.search import compare_frags
import numpy as np

@njit
def search_for_tag(query_frag, query_int, tag, reporter_frag_tolerance, ppm):
        
    report = np.zeros(len(tag))
    off_mass = np.zeros_like(tag)
    
    hits = compare_frags(query_frag, tag, reporter_frag_tolerance, ppm)
    for idx, _ in enumerate(hits):
        if _ > 0:
            report[idx] = query_int[_-1]
            off_mass[idx] = query_frag[_-1] - tag[idx]

            if ppm:
                off_mass[idx] = off_mass[idx] / (query_frag[_-1] + tag[idx]) *2 * 1e6
                    
    return report, off_mass

In [ ]:
def test_search_fort_tag():
    query_frag = np.array([1,2,3,4,5])
    query_int = np.array([1,2,3,4,5])
    tag = np.array([1,2,3,4,5])
    frag_tolerance = 0.1
    ppm= False

    assert (search_for_tag(query_frag, query_int, tag, frag_tolerance, ppm)[0] == query_int).all()

    query_frag = np.array([1,2,3,4,6])
    query_int = np.array([1,2,3,4,5])

    assert (search_for_tag(query_frag, query_int, tag, frag_tolerance, ppm)[0] == np.array([1,2,3,4,0])).all()
    
    query_frag = np.array([1,2,3,4,6])
    query_int = np.array([5,4,3,2,1])

    assert (search_for_tag(query_frag, query_int, tag, frag_tolerance, ppm)[0] == np.array([5,4,3,2,0])).all()
    
    
test_search_fort_tag()

## MS2 Search

In [ ]:
#export 
import alphapept.io

def search_tags_on_ms_file(file_name, tag, reporter_frag_tolerance, ppm):

    ms_file = alphapept.io.MS_Data_File(file_name, is_read_only = False)
    
    df = ms_file.read(dataset_name='peptide_fdr')
    tag_intensities = np.zeros((len(df), len(tag.channels)))
    tagged = df['sequence'].str.startswith(tag.mod_name).values
    query_data = ms_file.read_DDA_query_data()

    query_indices = query_data["indices_ms2"]
    query_frags = query_data['mass_list_ms2']
    query_ints = query_data['int_list_ms2']

    for idx, query_idx in enumerate(df['raw_idx']):

        query_idx_start = query_indices[query_idx]
        query_idx_end = query_indices[query_idx + 1]
        query_frag = query_frags[query_idx_start:query_idx_end]
        query_int = query_ints[query_idx_start:query_idx_end]

        query_frag_idx = query_frag < tag.masses[-1]+1
        query_frag = query_frag[query_frag_idx]
        query_int = query_int[query_frag_idx]

        if tagged[idx]:
            tag_int, off_mass = search_for_tag(query_frag, query_int, tag.masses, reporter_frag_tolerance, ppm)
            tag_intensities[idx, :] = tag_int

    df[tag.channels] = tag_intensities
    df[[_+'_off_ppm' for _ in tag.channels]] = off_mass
    
    ms_file.write(df, dataset_name="peptide_fdr", overwrite=True) #Overwrite dataframe with tag information


In [ ]:
#export
import logging
import os

from alphapept.constants import label_dict

def find_tags(
    to_process: dict,
    callback: callable = None,
    parallel:bool = False
) -> bool:
    """Wrapper function to convert raw to ms_data_file hdf.

    Args:
        to_process (dict): A dictionary with settings indicating which files are to be processed and how.
        callback (callable): A function that accepts a float between 0 and 1 as progress. Defaults to None.
        parallel (bool): If True, process multiple files in parallel.
            This is not implemented yet!
            Defaults to False.

    Returns:
        bool: True if and only if the tag finding was succesful.

    """
    index, settings = to_process
    raw_file = settings['experiment']['file_paths'][index]
    try:
        base, ext = os.path.splitext(raw_file)
        file_name = base+'.ms_data.hdf'
        tag = label_dict[settings['isobaric_label']['label']]
        
        reporter_frag_tolerance = settings['isobaric_label']['reporter_frag_tolerance']
        ppm = settings['isobaric_label']['reporter_frag_tolerance_ppm']
        
        search_tags_on_ms_file(file_name, tag, reporter_frag_tolerance, ppm)
        
        logging.info(f'Tag finding of file {file_name} complete.')
        return True
    except Exception as e:
        logging.error(f'Tag finding of file {file_name} failed. Exception {e}')
        return f"{e}" #Can't return exception object, cast as string
    return True

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_settings.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_interface.ipynb.
Converted 12_performance.ipynb.
Converted 13_export.ipynb.
Converted 14_display.ipynb.
Converted 15_label.ipynb.
Converted additional_code.ipynb.
Converted contributing.ipynb.
Converted file_formats.ipynb.
Converted index.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted Untitled2.ipynb.
Converted Untitled3.ipynb.
Converted Untitled4.ipynb.
Converted Untitled5.ipynb.
